### Ingestion del archivo "movie_genre.json"

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/common_functions"

In [0]:
%run "../includes/configuration"

#### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de spark

In [0]:
from pyspark.sql.types import *

In [0]:
# definimos el schema

movies_genres_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("genreId", IntegerType(), True)
])

In [0]:
movies_genres_df = spark.read \
            .schema(movies_genres_schema)\
            .json(f"{bronze_folder_path}/{v_file_date}/movie_genre.json")

#### Paso 2 - Cambiar el nombre de las columnas y añadir "ingestion_date" y "environment"

In [0]:
from pyspark.sql.functions import *

In [0]:
movies_genres_final_df = add_ingestion_date(movies_genres_df) \
                    .withColumnRenamed("movieId", "movie_id") \
                    .withColumnRenamed("genreId", "genre_id") \
                    .withColumn("environment", lit(v_environment)) \
                    .withColumn("file_date", lit(v_file_date))

In [0]:
display(movies_genres_final_df)

#### Paso 3 - Escribir la salida en formato "Parquet"

In [0]:
#overwrite_partition(movies_genres_final_df, "movie_silver", "movies_genres", "file_date")

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.genre_id = src.genre_id AND tgt.file_date = src.file_date'
merge_delte_lake(movies_genres_final_df, "movie_silver", "movies_genres", silver_folder_path, merge_condition, "file_date")

In [0]:
%sql
SELECT file_date, count(1)
FROM movie_silver.movies_genres
GROUP BY file_date;

In [0]:
%fs
ls /mnt/moviehistory2025/silver/movies_genres

In [0]:
dbutils.notebook.exit("success")